# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
cities_df = pd.read_csv("../WeatherPy/city_data.csv")
cities_df

,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,Toucheng,24.86,121.82,304.15,70,59,3.60,TW,1594570279
1,1,Busselton,-33.65,115.33,287.04,79,60,7.10,AU,1594570280
2,2,Bogoslovka,53.21,44.80,301.48,34,0,2.31,RU,1594570280
3,3,Hambantota,6.12,81.12,301.15,78,75,3.60,LK,1594570280
4,4,Upernavik,72.79,-56.15,278.75,83,99,4.37,GL,1594570254
...,...,...,...,...,...,...,...,...,...,...
541,541,Yerbogachën,61.28,108.01,286.90,79,1,1.31,RU,1594570350
542,542,Kodino,63.72,39.64,291.33,63,53,1.33,RU,1594570350
543,543,Plattsburgh,44.70,-73.45,299.15,65,75,3.10,US,1594570211
544,544,Belmonte,-15.86,-38.88,298.15,69,40,2.60,BR,1594570350


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
#Store latitude and longtitude in locations
gmaps.configure(api_key = g_key)
locations = cities_df[["Lat","Lng"]]
weight = cities_df["Humidity"]
fig = gmaps.figure(map_type = "HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations,weights = weight)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:

A max temperature lower than 30 degrees celsius (303 Kelvin) but higher than 25 degrees celsius (298 Kelvin).

Wind speed less than 10 mph.

Zero cloudiness.

Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [38]:
ideal_df = cities_df.loc[(cities_df["Max_Temp"]>298) & (cities_df["Max_Temp"]<303) & (cities_df["Wind_Speed"]<10) & (cities_df["Cloudiness"]==0)].reset_index()
ideal_df

,index,Unnamed: 0,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,2,2,Bogoslovka,53.21,44.80,301.48,34,0,2.31,RU,1594570280
1,64,64,Torbat-e Jām,35.24,60.62,300.57,16,0,5.69,IR,1594570287
2,68,68,Acajutla,13.59,-89.83,301.15,61,0,1.67,SV,1594570051
3,171,171,Urmary,55.68,47.94,300.15,34,0,2.00,RU,1594570302
4,184,184,Ponta do Sol,32.67,-17.10,298.15,66,0,1.04,PT,1594570304
5,280,280,Zadar,44.12,15.24,301.15,30,0,6.20,HR,1594570316
6,288,288,Várzea da Palma,-17.60,-44.73,302.64,28,0,1.58,BR,1594570317
7,298,298,Guerrero Negro,27.98,-114.06,299.63,44,0,6.35,MX,1594570319
8,341,341,Micheweni,-4.97,39.83,298.16,78,0,5.60,TZ,1594570325
9,342,342,Hauterive,46.09,3.45,301.15,30,0,2.60,FR,1594570325


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = ideal_df[["City","Country","Lat","Lng"]]
hotel_df

,City,Country,Lat,Lng
0,Bogoslovka,RU,53.21,44.80
1,Torbat-e Jām,IR,35.24,60.62
2,Acajutla,SV,13.59,-89.83
3,Urmary,RU,55.68,47.94
4,Ponta do Sol,PT,32.67,-17.10
5,Zadar,HR,44.12,15.24
6,Várzea da Palma,BR,-17.60,-44.73
7,Guerrero Negro,MX,27.98,-114.06
8,Micheweni,TZ,-4.97,39.83
9,Hauterive,FR,46.09,3.45


In [47]:
#set up parameters dictionary to update each iteration
params = {
    "radius":5000,
    "types":"hotel",
    "keyword":"hotel",
    "key":g_key
    
}

Hotel_Name = []

#use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    print(index)
    
    #get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat}, {lng}"
    
    #use the search term: "hotel" and lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request and print url
    name_address = requests.get(base_url,params=params)
    
    #convert to json
    name_address = name_address.json()
#     print(json.dumps(name_address2, indent = 4, sort_keys=True))
    
#remove exceptions
    try:
        Hotel_Name.append(name_address["results"][0]["name"])
    except (KeyError, IndexError):
        Hotel_Name.append("Hotel Not Found")

Hotel_Name

0
1
2
3
4
5
6
7
8
9
10


['Hotel Not Found',
 'Pardisan Hotel 🏩',
 'Villavela Hotel Boutique',
 'Hotel Not Found',
 'Estalagem Da Ponta Do Sol',
 'Villa Nico',
 'Hotel Casa Grande e Restaurante Forno e Fogão',
 'Hotel Los Caracoles',
 'Hotel Not Found',
 'VICHY CÉLESTINS SPA HÔTEL . Vichy - France',
 'Desert Gat']

In [48]:
hotel_df["Hotel Name"] = Hotel_Name
hotel_df

C:\Users\lluo\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Lat,Lng,Hotel Name
0,Bogoslovka,RU,53.21,44.80,Hotel Not Found
1,Torbat-e Jām,IR,35.24,60.62,Pardisan Hotel 🏩
2,Acajutla,SV,13.59,-89.83,Villavela Hotel Boutique
3,Urmary,RU,55.68,47.94,Hotel Not Found
4,Ponta do Sol,PT,32.67,-17.10,Estalagem Da Ponta Do Sol
5,Zadar,HR,44.12,15.24,Villa Nico
6,Várzea da Palma,BR,-17.60,-44.73,Hotel Casa Grande e Restaurante Forno e Fogão
7,Guerrero Negro,MX,27.98,-114.06,Hotel Los Caracoles
8,Micheweni,TZ,-4.97,39.83,Hotel Not Found
9,Hauterive,FR,46.09,3.45,VICHY CÉLESTINS SPA HÔTEL . Vichy - France


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))